<a href="https://colab.research.google.com/github/mdellanina/TrabajoPracticoCODERHOUSE/blob/main/TrabajoPracticoCH_DS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto final: "Modelo para la Predicción de Nivel de Radiación Solar"
## Profesor: Jorge Ruiz
## Tutor: Facundo Juliá
## Alumno: Mariano Della Nina


# 1. Introducción

El conjunto de datos contiene columnas como: "dirección del viento", "velocidad del viento", "humedad" y "temperatura".
El parámetro de respuesta que se va a predecir es: "Radiación_solar".
Contiene mediciones de los últimos 4 meses y hay que predecir el nivel de radiación solar.





## 1.1 About DataSet

Estos conjuntos de datos son datos meteorológicos de la estación meteorológica HI-SEAS de cuatro meses (septiembre a diciembre de 2016) entre la Misión IV y la Misión V.

Para cada conjunto de datos, los campos son:

*   Un número de fila (1-n) útil para ordenar los resultados de esta exportación.
*   La fecha time_t de UNIX (segundos desde el 1 de enero de 1970). Útil para ordenar los resultados de esta exportación con los resultados de otras exportaciones
*   La fecha en formato aaaa-mm-dd
*   La hora local del día en formato hh:mm:ss de 24 horas.
*   Los datos numéricos, si los hay (puede ser una cadena vacía)
*   Los datos de texto, si los hay (puede ser una cadena vacía)

Las unidades de cada conjunto de datos son:

*   Radiación solar: w/m^2 (watts/metro^2)
*   Temperatura: °F (grados Fahrenheit)
*   Humedad: % (porcentaje)
*   Presión barométrica: Hg (pulgada de mercurio)
*   Dirección del viento: grados
*   Velocidad del viento: mph (millas por hora)
*   Amanecer/atardecer: hora de Hawái

## 1.2 Dependencias

In [15]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import boxplot, distplot

## 1.3 DataSet

In [17]:
# NO anduvo buscarlo de GITHUB directamente con:
# df_solar = pd.read_csv("https://raw.githubusercontent.com/marianodn/proyecto_final_radiaccion_solar/master/solar_radiation.csv")
path = '/content/SolarPrediction.csv'
df = pd.read_csv(path)
display(df)

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [76]:
print(pd.to_datetime(df.UNIXTime, format = '%d-%m-%Y'))

ValueError: time data "1475229326" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

# 2. Estadistica Descriptiva

## 2.1 Tamaño de la muestra

In [32]:
print("Este dataSet tiene la siguiente cantidad de filas y columnas: " + str(df.shape))

Este dataSet tiene la siguiente cantidad de filas y columnas: (32686, 11)


El dataset obtenido de la base de datos posee 32686 filas y 11 columnas, de las cuales 10 son variables independientes a analizar y 1 corresponde a la variable target ("Radiation")

## 2.2 Integridad de Datos

In [74]:
print("****************************************")
print("Analisis de valores duplicados: ")
print("****************************************")
print(df.duplicated().value_counts())
if "df.duplicated().value_counts() == True":
  print("\n***** No hay datos duplicados *****")
else:
  print("\n***** Hay datos duplicados *****")
print("\n****************************************")
print("Analisis de existencia de valores nulos: ")
print("****************************************")
df.info(verbose = True)
if "df.isna().value_counts() == True":
  print("\n***** No hay valores nulos *****\n")
else:
  print("\n***** Hay valores nulos *****\n")

****************************************
Analisis de valores duplicados: 
****************************************
False    32686
Name: count, dtype: int64

***** No hay datos duplicados *****

****************************************
Analisis de existencia de valores nulos: 
****************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Data                    32686 non-null  object 
 2   Time                    32686 non-null  object 
 3   Temperature             32686 non-null  int64  
 4   Pressure                32686 non-null  float64
 5   Humidity                32686 non-null  int64  
 6   WindDirection(Degrees)  32686 non-null  float64
 7   Speed                   32686 non-null  float64
 8   TimeSunRise             32686 non-null

In [75]:
df.describe()

,UNIXTime,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,Radiation
count,3.268600e+04,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000
mean,1.478047e+09,51.103255,30.422879,75.016307,143.489821,6.243869,207.124697
std,3.005037e+06,6.201157,0.054673,25.990219,83.167500,3.490474,315.916387
min,1.472724e+09,34.000000,30.190000,8.000000,0.090000,0.000000,1.110000
25%,1.475546e+09,46.000000,30.400000,56.000000,82.227500,3.370000,1.230000
50%,1.478026e+09,50.000000,30.430000,85.000000,147.700000,5.620000,2.660000
75%,1.480480e+09,55.000000,30.460000,97.000000,179.310000,7.870000,354.235000
max,1.483265e+09,71.000000,30.560000,103.000000,359.950000,40.500000,1601.260000
